<a href="https://colab.research.google.com/github/mJekal/LG_Aimers/blob/main/data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re
import pandas as pd

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 200)

import os, sys
from google.colab import drive
drive.mount('/content/gdrive')

default_dir = "/content/gdrive/My Drive"
df_train = pd.read_csv(os.path.join(default_dir,"train.csv")) # 학습용 데이터
df_test = pd.read_csv(os.path.join(default_dir,"submission.csv")) # 테스트 데이터(제출파일의 데이터)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### 데이터 처리 함수

In [2]:
def process_country_data(df):
    df['customer_country'] = df['customer_country'].str.upper()
    split_data = df['customer_country'].str.split('/', expand=True)
    df['City'] = split_data[1].str.strip()
    df['Country'] = split_data[2].str.strip()

def clean_strings(df, columns):
    for column in columns:
        df[column] = df[column].str.lower().replace(r'[^a-zA-Z0-9]', '', regex=True)

def replace_missing_values(df_train, df_test, column, replace_value):
    missing_values = df_test[~df_test[column].isin(df_train[column])][column]
    df_test.loc[df_test[column].isin(missing_values), column] = replace_value

def calculate_ver_country(df):
    df['ver_country'] = df['com_reg_ver_win_rate'] / df['ver_win_ratio_per_bu']

def preprocess_str_columns(df, column_names):
    for column_name in column_names:
        df[column_name] = df[column_name].str.replace(" ", "").str.lower()

def map_category(value, mapping_dict):
    for category, values in mapping_dict.items():
        if value in values:
            return category
    return value  # 매핑되지 않은 경우 원래 값을 반환

def apply_mapping(df, column, mapping_dict):
    df[column] = df[column].map(lambda x: map_category(x, mapping_dict))

def fill_missing_values(df_train, df_test, columns, fill_value="NONE"):
    for column in columns:
        df_train[column].fillna(fill_value, inplace=True)
        df_test[column].fillna(fill_value, inplace=True)

def update_column(df, business_unit, column):
    cond = (
        (df['business_unit'] == business_unit) &
        ((df['business_area'] == 'corporate / office') | (df['business_area'] == 'hotel & accommodation')) &
        df[column].isna()
    )
    df.loc[cond, column] = 1
    df[column].fillna(0, inplace=True)

def generalize_column_values(df, column, condition_dict):
    for condition_value, replace_value in condition_dict.items():
        rows = df[df[column] == condition_value].index
        df.loc[rows, column] = replace_value

def map_conversion_rates(df, column):
    conversion_rates = df.groupby(column)['is_converted'].mean() * 100
    df[column] = df[column].map(conversion_rates)


### 매핑

In [3]:
product_mapping = { 'HVAC/ESS': ['control','multiinverter','arcondicionadoresidencial','tetooucasseteinverter', 'ventilation', 'vrf', 'multi-split', 'single-split','rac', 'ess','chiler','chiller', 'heating'],
                     'Commercial Display': ['highbrightness','lgone:quickseries','interactivedigitalboard','oledsignage', 'ledsignage', 'videowallsignage', 'interactivesignage', 'highbrightnesssignage', 'specialsignage', 'standardsignage', 'hoteltv', 'hospitaltv', 'softwaresolution', 'signagecaresolution', 'accessories', 'webos', 'one:quickseries', 'pro:centric'],
                      'IT PRODUCTS': ['monitor', 'laptop', 'projector', 'clouddevice','notebook', 'medicaldisplay'],
                      'Commerical Laundry': ['titan(largecapacity)', 'giant(standardcapacity)'],
                      'Compressor & Motor': ['reciprocatingcompressor', 'rotarycompressor', 'scrollcompressor', 'motor'],
                      'ADVANCED MATERIALS': ['antimicrobial', 'porcelainenamel', 'specialtyglass'] ,
                      'Robot': ['lgcloiuv-cbot', 'lgcloiservebot(shelftype)', 'lgcloiservebot(drawertype)', 'lgcloiguidebot'],
                      'Others':['etc.','others','other']
                    }

customer_type_mapping = { 'End Customer': ['endcustomer','end-user','end-customer', 'constructioncompany', 'owner/developer', 'medical/healthcarefacility', 'government/publicsector',  'corporate', 'education', 'retail', 'fitness'],
                         'Channel Partner' : ['agent','channelpartner', 'distributor','reseller', 'nsp(usonly)', 'nationalreseller', 'regionalreseller', 'si(systemintegrator)', 'proav/avconsultant', 'var(3po)'],
                          'Specifier/ Influencer': ['specifier/influencer','architect', 'consultant', 'contractor', 'technical/designfirm', 'regionbuilder', 'installer', 'ad&contentsprovider', 'appliedrep'],
                          'Solution Eco-Partner': ['solutioneco-partner','cms/webos/isv', 'mount/metalfabrication','meetingsolution', 'control/processor', 'externalcompute'],
                           'Developer': ['developer'],
                          'Service Partner': ['servicepartner','authorizedservicecenter', 'authorizedservicedealer'], }

continent_mapping = { 'Europe': ['LGEWA', 'LGEMA', 'LGEWR', 'LGEUK', 'LGEFS', 'LGEES', 'LGEEH', 'LGEJE', 'LGEDG', 'LGEIS', 'LGEMK', 'LGEPL', 'LGESW', 'LGEHS', 'LGEAG', 'LGERO', 'LGECZ', 'LGEPT', 'LGEBN', 'LGESC', 'LGELS', 'LGENO','LGEMF'],
                        'CSI' : ['LGEUA', 'LGEAK','LGERU', 'LGERA', 'LGERI', 'LGERM', 'LGEUR', 'LGELV'],
                        'China': ['LGETR', 'LGERD', 'LGEHZ', 'LGEND', 'LGEHK', 'LGETT',  'LGEPN', 'LGECH','LGEQH', 'LGESH', 'LGESY', 'LGETA', 'LGEYT', 'LGEKS', 'LGENP', 'LGEHN', 'LGEQD'],
                       'Asia': ['LGEIL', 'LGSI','LGEAP',  'LGETH', 'LGEVN', 'LGEIN', 'LGESL', 'LGEML', 'LGEJP', 'LGEPH', 'LGEVH','LGEKR'],
                       'MIDDLE EAST & America': ['LGEEG', 'LGEAT', 'LGESR', 'LGETK', 'LGESA', 'LGEMC', 'LGEGF', 'LGEME', 'LGEOT', 'LGEEC', 'LGENI', 'LGEAF', 'LGELF','LGESJ'],
                       'North America': ['LGEMX', 'LGEMM', 'LGEAI', 'LGECI', 'LGEUS', 'LGEMU', 'LGEMS', 'LGEMR'],
                       'SOUTH & CENTRAL AMERICA' :['LGEAR', 'LGEAZ', 'LGECB', 'LGECL', 'LGEPR', 'LGEPS','LGERS', 'LGESP' ],
 }

**슬래시 기준으로 City와 Country 새로운 칼럼 만들기**

In [4]:
def process_country_data(df):
    df['customer_country'] = df['customer_country'].str.upper()
    split_data = df['customer_country'].str.split('/', expand=True)
    df['City'] = split_data[1].str.strip()
    df['Country'] = split_data[2].str.strip()

process_country_data(df_train)
process_country_data(df_test)

**알파벳이 아니거나 숫자가 아닌 문자를 제거하고, 모든 문자를 소문자로 변환**

변환할 칼럼 : City, Country,expected_timeline,customer_job,inquiry_type,customer_position

In [5]:
def clean_strings(df, columns):
    for column in columns:
        df[column] = df[column].str.lower().replace(r'[^a-zA-Z0-9]', '', regex=True)

columns_to_clean = ['City', 'Country','expected_timeline','customer_job','inquiry_type','customer_position']

clean_strings(df_train, columns_to_clean)
clean_strings(df_test, columns_to_clean)

**Test데이터 셋에서 Train 데이터 셋에 없는 국가를 "None"으로 대체**

In [6]:
def replace_missing_values(df_train, df_test, column, replace_value):
    missing_values = df_test[~df_test[column].isin(df_train[column])][column]
    df_test.loc[df_test[column].isin(missing_values), column] = replace_value

replace_missing_values(df_train, df_test, 'Country', "None")

**Vertical Level 1과 Business Unit에서 특정 Region의 가중 영업 전환된 샘플 비율(?)**

In [7]:
def calculate_ver_country(df):
    df['ver_country'] = df['com_reg_ver_win_rate'] / df['ver_win_ratio_per_bu']

calculate_ver_country(df_train)
calculate_ver_country(df_test)

**Test데이터 셋에서 Train 데이터 셋에 없는 ID를 99999으로 대체**

In [8]:
replace_missing_values(df_train, df_test, 'customer_idx', 99999)

**문자열에서 공백을 제거하고 모든 문자열을 소문자로 변환**

In [9]:
def preprocess_str_columns(df, column_names):
    for column_name in column_names:
        df[column_name] = df[column_name].str.replace(" ", "").str.lower()

columns_to_preprocess = ['customer_type', 'product_category']

preprocess_str_columns(df_train, columns_to_preprocess)
preprocess_str_columns(df_test, columns_to_preprocess)

**LG inquiry-to-buy 참고해서 재범주화**

적용할 칼럼 : customer_type, product_category

In [10]:
def map_category(value, mapping_dict):
    for category, values in mapping_dict.items():
        if value in values:
            return category
    return value  # 매핑되지 않은 경우 원래 값을 반환

def apply_mapping(df, column, mapping_dict):
    df[column] = df[column].map(lambda x: map_category(x, mapping_dict))

product_mapping = { 'HVAC/ESS': ['control','multiinverter','arcondicionadoresidencial','tetooucasseteinverter', 'ventilation', 'vrf', 'multi-split', 'single-split','rac', 'ess','chiler','chiller', 'heating'],
                     'Commercial Display': ['highbrightness','lgone:quickseries','interactivedigitalboard','oledsignage', 'ledsignage', 'videowallsignage', 'interactivesignage', 'highbrightnesssignage', 'specialsignage', 'standardsignage', 'hoteltv', 'hospitaltv', 'softwaresolution', 'signagecaresolution', 'accessories', 'webos', 'one:quickseries', 'pro:centric'],
                      'IT PRODUCTS': ['monitor', 'laptop', 'projector', 'clouddevice','notebook', 'medicaldisplay'],
                      'Commerical Laundry': ['titan(largecapacity)', 'giant(standardcapacity)'],
                      'Compressor & Motor': ['reciprocatingcompressor', 'rotarycompressor', 'scrollcompressor', 'motor'],
                      'ADVANCED MATERIALS': ['antimicrobial', 'porcelainenamel', 'specialtyglass'] ,
                      'Robot': ['lgcloiuv-cbot', 'lgcloiservebot(shelftype)', 'lgcloiservebot(drawertype)', 'lgcloiguidebot'],
                      'Others':['etc.','others','other']
                    }

customer_type_mapping = { 'End Customer': ['endcustomer','end-user','end-customer', 'constructioncompany', 'owner/developer', 'medical/healthcarefacility', 'government/publicsector',  'corporate', 'education', 'retail', 'fitness'],
                         'Channel Partner' : ['agent','channelpartner', 'distributor','reseller', 'nsp(usonly)', 'nationalreseller', 'regionalreseller', 'si(systemintegrator)', 'proav/avconsultant', 'var(3po)'],
                          'Specifier/ Influencer': ['specifier/influencer','architect', 'consultant', 'contractor', 'technical/designfirm', 'regionbuilder', 'installer', 'ad&contentsprovider', 'appliedrep'],
                          'Solution Eco-Partner': ['solutioneco-partner','cms/webos/isv', 'mount/metalfabrication','meetingsolution', 'control/processor', 'externalcompute'],
                           'Developer': ['developer'],
                          'Service Partner': ['servicepartner','authorizedservicecenter', 'authorizedservicedealer'], }

apply_mapping(df_train, 'product_category', product_mapping)
apply_mapping(df_test, 'product_category', product_mapping)
apply_mapping(df_train, 'customer_type', customer_type_mapping)
apply_mapping(df_test, 'customer_type', customer_type_mapping)

**결측치를 지정한 "NONE"으로 채우기**

In [11]:
def fill_missing_values(df_train, df_test, columns, fill_value="NONE"):
    for column in columns:
        df_train[column].fillna(fill_value, inplace=True)
        df_test[column].fillna(fill_value, inplace=True)

columns_to_fill = ['customer_type', 'expected_timeline']

fill_missing_values(df_train, df_test, columns_to_fill)

**결측치가 조건에 맞으면 1, 아니면 0**

적용할 칼럼 : id_strategic_ver, it_strategic_ver



In [12]:
def update_column(df, business_unit, column):
    cond = (
        (df['business_unit'] == business_unit) &
        ((df['business_area'] == 'corporate / office') | (df['business_area'] == 'hotel & accommodation')) &
        df[column].isna()
    )
    df.loc[cond, column] = 1
    df[column].fillna(0, inplace=True)

update_column(df_train, 'ID',  'id_strategic_ver')
update_column(df_test, 'ID', 'id_strategic_ver')
update_column(df_train, 'IT',  'it_strategic_ver')
update_column(df_test, 'IT', 'it_strategic_ver')

**id_strategic_ver가 1이거나 it_strategic_ver가 1이면 결측치 1, 아니면 0**

In [13]:
df_train['idit_strategic_ver'] = df_train.apply(lambda row: 1 if row['id_strategic_ver'] == 1 or row['it_strategic_ver'] == 1 else 0, axis=1)
df_test['idit_strategic_ver'] = df_test.apply(lambda row: 1 if row['id_strategic_ver'] == 1 or row['it_strategic_ver'] == 1 else 0, axis=1)

**특정 조건에 따라 값들을 대체**

적용할 칼럼 : customer_job, inquiry_type

In [14]:
def generalize_column_values(df, column, condition_dict):
    for condition_value, replace_value in condition_dict.items():
        rows = df[df[column] == condition_value].index
        df.loc[rows, column] = replace_value

condition_dict_test = {'healthcare': 'healthcareservices'}
generalize_column_values(df_test, 'customer_job', condition_dict_test)
condition_dict_train = {'purchaseorquotation': 'usageortechnicalconsultation'}
generalize_column_values(df_train, 'inquiry_type', condition_dict_train)

**평균 변환율을 계산하고 변환율을 적용**

In [15]:
def map_conversion_rates(df, column):
    conversion_rates = df.groupby(column)['is_converted'].mean() * 100
    df[column] = df[column].map(conversion_rates)

map_conversion_rates(df_train, 'lead_owner')

**사업본부로 대륙 매핑**

In [16]:
continent_mapping = { 'Europe': ['LGEWA', 'LGEMA', 'LGEWR', 'LGEUK', 'LGEFS', 'LGEES', 'LGEEH', 'LGEJE', 'LGEDG', 'LGEIS', 'LGEMK', 'LGEPL', 'LGESW', 'LGEHS', 'LGEAG', 'LGERO', 'LGECZ', 'LGEPT', 'LGEBN', 'LGESC', 'LGELS', 'LGENO','LGEMF'],
 'CSI' : ['LGEUA', 'LGEAK','LGERU', 'LGERA', 'LGERI', 'LGERM', 'LGEUR', 'LGELV'],
 'China': ['LGETR', 'LGERD', 'LGEHZ', 'LGEND', 'LGEHK', 'LGETT',  'LGEPN', 'LGECH','LGEQH', 'LGESH', 'LGESY', 'LGETA', 'LGEYT', 'LGEKS', 'LGENP', 'LGEHN', 'LGEQD'],
 'Asia': ['LGEIL', 'LGSI','LGEAP',  'LGETH', 'LGEVN', 'LGEIN', 'LGESL', 'LGEML', 'LGEJP', 'LGEPH', 'LGEVH','LGEKR'],
 'MIDDLE EAST & America': ['LGEEG', 'LGEAT', 'LGESR', 'LGETK', 'LGESA', 'LGEMC', 'LGEGF', 'LGEME', 'LGEOT', 'LGEEC', 'LGENI', 'LGEAF', 'LGELF','LGESJ'],
 'North America': ['LGEMX', 'LGEMM', 'LGEAI', 'LGECI', 'LGEUS', 'LGEMU', 'LGEMS', 'LGEMR'],
 'SOUTH & CENTRAL AMERICA' :['LGEAR', 'LGEAZ', 'LGECB', 'LGECL', 'LGEPR', 'LGEPS','LGERS', 'LGESP' ],
 }

df_train['Continent'] = df_train['response_corporate'].map({value: continent for continent, values in continent_mapping.items() for value in values})
df_test['Continent'] = df_test['response_corporate'].map({value: continent for continent, values in continent_mapping.items() for value in values})


### EDA

##1 bant_submit
결측치 없다
##2 customer_country
customer_country.1과 동일

슬래시 기준으로 City와 Country 새로운 칼럼 만들 수 있음

결측치 0.016560 -> NaN으로 나둠

특수문자 제거, 소문자

테스트 셋에 있는 데 train 셋에 없으면 None

##3 business_unit
결측치 없다

##4 com_reg_ver_win_rate
"com_reg_ver_win_rate"에서는 특정 지역의 전체 영업 전환된 샘플 비율을 나타냄

"ver_win_ratio_per_bu"는 특정 Vertical Level 1의 Business Unit 별 영업 전환된 샘플 비율을 나타 냄

이 두 값을 나누면 특정 Vertical Level 1과 Business Unit에서 특정 Region의 가중 영업 전환된 샘플 비율을 구할 수 있을것이라 생각

결측치 -> 그대로 -> miceforest

ver_country 칼럼 만들기

##5 customer_idx
결측치 없다

테스트 셋에 있는 데 train 셋에 없으면 99999
##6 customer_type
LG inquiry-to-buy 참고해서 재범주화

결측치 : 0.741345 -> "NONE"으로 바꾸기

##7 enterprise
결측치 없다

원핫 인코딩
##8 historical_existing_cnt
결측치 비율 0.768023 -> miceforest
##9 id_strategic_ver
business_area가 corporate / office 또는 hotel & accomodataion 이고 business_unit이 ID면 1

결측치 위 조건에 해당하면 1, 해당하지 않으면 0
##10 it_strategic_ver
business_area가 corporate / office 또는 hotel & accomodataion 이고 business_unit이 IT면 1

결측치 위 조건에 해당하면 1, 해당하지 않으면 0
##11 idit_strategic_ver
id_strategic_ver 또는 it_strategic_ver 중 하나라도 1이면 1

결측치 위 조건에 해당하면 1, 해당하지 않으면 0
##12 customer_job
결측치 비율 : 0.315908 -> 그대로

특수문자 제거, 소문자 변경

test셋의 healthcare를 healthcareservices로

##13 lead_desc_length
skewed 되어 있다 -> Min-Max 스케일

결측치 없음
##14 inquiry_type
결측치 : 0.015869 -> 그대로 나둔다

특수문자 제거, 소문자 변경

train 셋의 purchaseorquotation를 usageortechnicalconsultation로

##15 product_category
LG inquiry-to-buy 참고해서 재범주화

결측치 0.326717 -> 그대로

##16 product_subcategory
결측치 비율 0.8442638155786776 -> 컬럼 삭제
##17 product_modelname
결측치 비율 0.8443649977234018 -> 컬럼 삭제
##18 customer_country.1
customer_country와 동일 -> 컬럼삭제
##19 cusomer_position
결측치 없음

특수문자 제거, 소문자 변경
##20 response_corporate
LG inquiry-to-buy 참고해서 재범주화

결측치 없음
##21 expected_timeline
결측치 0.520464 -> "NONE" 으로 변경
##22 ver_cus
결측치 없음

ver_cus와 business_area, customer_type과 밀접한 관련

ver_cus가 1인데 business_area나 customer_type이 결측치인 경우는 없다

ver_cus가 1일때 business_area type :

hotel & accommodation

retail

education

corporate / office

##23 ver_pro
결측치 없음

business_area와 product_category와 밀접한 관련

Product로 매핑한 결과, ver_pro가 1인 Product는 모두 Commercial Display에 속했다

##24 ver_win_rate_x

business_area에 따라 값 매칭

결측치 동일하고, business_area가 같으면 ver_win_rate_x가 같은 값

결측치 : ver_win_rate_x -> miceforest

##25 ver_win_ratio_per_bu
결측치 비율 : 0.741918 -> miceforest or 컬럼삭제

'business_area', 'business_unit'와 밀접하게 관련

'business_area'와 'business_unit'이 같으면 같은 값

'business_area'가 결측치 이면 'ver_win_ratio_per_bu'도 결측치 이다

'business_area'와 'business_unit' 둘 다 존재하는데 NaN값도 있다.

칼럼 삭제?

##26 business_area

결측치 0.689421 -> 컬럼 삭제

business_area가 결측치 이면, ver_cus와 ver_pro도 0 이다.

##27 business_subarea
business_subarea 0.906811 -> 컬럼 삭제

business_area가 결측치 이면, business_subarea도 결측치 이다.

##28 lead_owner
결측치 없음

Frequency Encoding

##삭제할 칼럼

customer_country

com_reg_ver_win_rate

customer_type

business_area

business_subarea

product_subcategory

product_modelname

customer_country.1


##새로 만든 칼럼
ver_country

City

Country

## LG Inquiry-to-buy 참고 해서 바꾼 칼럼
customer_type

product_category

response_corporate